In [1]:
    
    import jupyter_client, IPython, ipykernel.ipkernel, ipykernel.kernelapp, pidgy, traitlets, pidgy, traitlets, ipykernel.kernelspec, ipykernel.zmqshell, pathlib, traitlets

In [2]:
# The `pidgy` shell-kernel model

# The `pidgy` shell-kernel model

The shell is the application either jupyterlab or jupyter notebook, the kernel determines the programming language.  Below we design a just jupyter kernel that can be installed using 

    !pidgy kernel install

In [3]:

    class pidgyInteractiveShell(ipykernel.zmqshell.ZMQInteractiveShell):
Configure a native `pidgy` `IPython.InteractiveShell`

        weave = traitlets.Any()
        tangle = traitlets.Any()
        test = traitlets.Any()
        enable_html_pager = traitlets.Bool(True)
`pidgyInteractiveShell.enable_html_pager` is necessary to see rich displays in the inspector.
        
        def __init__(self,*args, **kwargs):
            super().__init__(*args, **kwargs)
            from .. import load_ipython_extension
            load_ipython_extension(self)
            
    class pidgyKernel(ipykernel.ipkernel.IPythonKernel):
        shell_class = traitlets.Type(pidgyInteractiveShell)
        def do_inspect(self, code, cursor_pos, detail_level=0):
`pidgyKernel.do_inspect` will default to wysiwyg configuration thats displays a preview
of the source input rendered as markdown.
            
            object = {'found': False}
            try: 
                object = super().do_inspect(code, cursor_pos, detail_level=0)
            except: ...
                
            if not object['found']:
Simulate finding an object and return a preview of the markdown.

                object['found'] = True
                line, offset = IPython.utils.tokenutil.line_at_cursor(code, cursor_pos)
                lead = code[:cursor_pos]
                col = cursor_pos - offset
                code = code[:cursor_pos]+F"""<sub>{len(lead.splitlines()) + int(not(col))}</sub><sup>{col + 1}</sup>"""+code[cursor_pos:]
                    
                object['data'] = {'text/markdown': code}
We include the line number and cursor position to enrich the connection between the inspector and the source code displayed on another part of the screen.

            return object